In [27]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
llm = ChatOpenAI(model="gpt-4o-mini")

llm.invoke([HumanMessage("잘 지냈어?")])

AIMessage(content='네, 잘 지냈습니다! 당신은 어떻게 지내고 계신가요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 12, 'total_tokens': 30, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CU4P2rQD1zq3ZLsQzQfrnIgECvhho', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--12d56b53-d370-41b4-acdf-f066a7bfbb00-0', usage_metadata={'input_tokens': 12, 'output_tokens': 18, 'total_tokens': 30, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [28]:
from langchain_core.tools import tool
from datetime import datetime
import pytz

@tool # @tool 데코레이터를 사용하여 함수를 도구로 등록
def get_current_time(timezone: str, location: str) -> str:
    """ 현재 시각을 반환하는 함수

    Args:
        timezone (str): 타임존 (예: 'Asia/Seoul') 실제 존재하는 타임존이어야 함
        location (str): 지역명. 타임존이 모든 지명에 대응되지 않기 때문에 이후 llm 답변 생성에 사용됨
    """
    tz = pytz.timezone(timezone)
    now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    location_and_local_time = f'{timezone} ({location}) 현재시각 {now} ' # 타임존, 지역명, 현재시각을 문자열로 반환
    print(location_and_local_time)
    return location_and_local_time


In [29]:
# 도구를 tools 리스트에 추가하고, tool_dict에도 추가
tools = [get_current_time,]
tool_dict = {"get_current_time": get_current_time,}

# 도구를 모델에 바인딩: 모델에 도구를 바인딩하면, 도구를 사용하여 llm 답변을 생성할 수 있음
llm_with_tools = llm.bind_tools(tools)

In [30]:
from langchain_core.messages import SystemMessage

# (4) 사용자의 질문과 tools 사용하여 llm 답변 생성
messages = [
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    HumanMessage("부산은 지금 몇시야?"),
]

# (5) llm_with_tools를 사용하여 사용자의 질문에 대한 llm 답변 생성
response = llm_with_tools.invoke(messages)
messages.append(response)

# (6) 생성된 llm 답변 출력
print(messages)

[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 135, 'total_tokens': 158, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CU4P7LePUR46rZe8cQFlykcE8JjiG', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--9dae6cc7-8396-48a9-8472-8939e1f77f32-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '부산'}, 'id': 'call_SIbomO7Kzf9xUyZBWx3haWfu', 'type': 'tool_call'}],

In [31]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]] # (7) tool_dict를 사용하여 도구 함수를 선택
    print(tool_call["args"]) # (8) 도구 호출 시 전달된 인자 출력
    tool_msg = selected_tool.invoke(tool_call) # (9) 도구 함수를 호출하여 결과를 반환
    messages.append(tool_msg)

messages

{'timezone': 'Asia/Seoul', 'location': '부산'}
Asia/Seoul (부산) 현재시각 2025-10-24 14:13:26 


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 135, 'total_tokens': 158, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CU4P7LePUR46rZe8cQFlykcE8JjiG', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--9dae6cc7-8396-48a9-8472-8939e1f77f32-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '부산'}, 'id': 'call_SIbomO7Kzf9xUyZBWx3haWfu', 'type': 'tool_call'}

In [32]:
llm_with_tools.invoke(messages)

AIMessage(content='부산은 지금 2025년 10월 24일 오후 2시 13분입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 192, 'total_tokens': 216, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CU4PABUTaPH4Ssu0lXbicjuxmqoUG', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--f25bd7f3-48ce-4fcc-951a-83d2b94a5298-0', usage_metadata={'input_tokens': 192, 'output_tokens': 24, 'total_tokens': 216, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [33]:
from pydantic import BaseModel, Field

class StockHistoryInput(BaseModel):
    ticker: str = Field(..., title="주식 코드", description="주식 코드 (예: AAPL)")
    period: str = Field(..., title="기간", description="주식 데이터 조회 기간 (예: 1d, 1mo, 1y)")


In [34]:
import yfinance as yf

@tool
def get_yf_stock_history(stock_history_input: StockHistoryInput) -> str:
    """ 주식 종목의 가격 데이터를 조회하는 함수"""
    stock = yf.Ticker(stock_history_input.ticker)
    history = stock.history(period=stock_history_input.period)
    history_md = history.to_markdown() 

    return history_md

tools = [get_current_time, get_yf_stock_history]
tool_dict = {"get_current_time": get_current_time, "get_yf_stock_history": get_yf_stock_history}

llm_with_tools = llm.bind_tools(tools)

In [35]:
messages.append(HumanMessage("테슬라는 한달 전에 비해 주가가 올랐나 내렸나?"))
               
response = llm_with_tools.invoke(messages)
print(response)
messages.append(response)

content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 283, 'total_tokens': 310, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CU4PGMGdomP0j1tL6fRrWYv0E3xqW', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--26324651-5163-4167-97f1-c6f13d8a8a03-0' tool_calls=[{'name': 'get_yf_stock_history', 'args': {'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}, 'id': 'call_4g5Jmtf7jgwSXUyc76i37r2n', 'type': 'tool_call'}] usage_metadata={'input_tokens': 283, 'output_tokens': 27, 'total_tokens': 310, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}

In [36]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]]
    print(tool_call["args"])
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    print(tool_msg)

{'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}
content='| Date                      |   Open |   High |    Low |   Close |      Volume |   Dividends |   Stock Splits |\n|:--------------------------|-------:|-------:|-------:|--------:|------------:|------------:|---------------:|\n| 2025-09-24 00:00:00-04:00 | 429.83 | 444.21 | 429.03 |  442.79 | 9.31336e+07 |           0 |              0 |\n| 2025-09-25 00:00:00-04:00 | 435.24 | 435.35 | 419.08 |  423.39 | 9.67464e+07 |           0 |              0 |\n| 2025-09-26 00:00:00-04:00 | 428.3  | 440.47 | 421.02 |  440.4  | 1.01628e+08 |           0 |              0 |\n| 2025-09-29 00:00:00-04:00 | 444.35 | 450.98 | 439.5  |  443.21 | 7.94915e+07 |           0 |              0 |\n| 2025-09-30 00:00:00-04:00 | 441.52 | 445    | 433.12 |  444.72 | 7.4358e+07  |           0 |              0 |\n| 2025-10-01 00:00:00-04:00 | 443.8  | 462.29 | 440.75 |  459.46 | 9.81223e+07 |           0 |              0 |\n| 2025-10-02 00:00:00-04:0

In [37]:
llm_with_tools.invoke(messages)

AIMessage(content='한 달 전인 2025년 9월 24일의 테슬라(TSLA) 주가는 **442.79** 달러로 마감했습니다. 현재(2025년 10월 23일) 주가는 **448.98** 달러입니다.\n\n따라서, 테슬라의 주가는 한 달 전보다 **상승**했습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 1643, 'total_tokens': 1723, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CU4PKVThC4IlaoFsXeJ5gisyLpzQ4', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--173273b3-8004-44ec-b244-1d1fbc40508c-0', usage_metadata={'input_tokens': 1643, 'output_tokens': 80, 'total_tokens': 1723, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})